In [2]:
import re
import numpy as np
import pandas as pd
import csv
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import sentiwordnet as swn #importing the sentiwordnet
from nltk.tag.perceptron import PerceptronTagger #importing a tagger


In [8]:
tagger=PerceptronTagger() #init the tagger in default mode
lemmatizer = WordNetLemmatizer()

features_df = pd.DataFrame(columns=['body','pw', 'nw', 'PW', 'NW', 'P', 'rep', 'excl', 'ques', 'dots', 'cap', 'quotes',\
                                    'score'])


#FOR TWITTER
sarcasmReddit_df = pd.read_csv('../input/twitter/semeval_sarcastic.csv',encoding = "ISO-8859-1")
seriousReddit_df = pd.read_csv('../input/twitter/non_sarcastic.csv',encoding = "ISO-8859-1")


In [9]:

############# Cleaned dataframe ##############
frames = [sarcasmReddit_df, seriousReddit_df]
alldata = pd.concat(frames).reset_index(drop=True)

############# Drop empty cells #############
alldata['body'].replace('', np.nan, inplace=True)
alldata.dropna(subset=['body'], inplace=True)


In [10]:
delta = 3

def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

def sentiment_score(word, tag, flag):
    sentiment = 0
    wn_tag = penn_to_wn(tag)
    if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
        return sentiment
    
    lemma = lemmatizer.lemmatize(word, pos=wn_tag)
    if not lemma:
        return sentiment

    synsets = wn.synsets(lemma, pos=wn_tag)
    if not synsets:
        return sentiment
    # Take the first sense, the most common
    synset = synsets[0]
    swn_synset = swn.senti_synset(synset.name())
 
    sentiment += swn_synset.pos_score() - swn_synset.neg_score()
    #print("sentiment of ",word, " is ", sentiment)#added to check
    if flag == 0:
        return sentiment
    elif flag == 1:
        if sentiment > 0:
            if wn_tag not in (wn.VERB, wn.ADJ, wn.ADV):
                return 0
            else:
                return 1
        else:
            return 0
    elif flag == 2:
        if sentiment < 0:
            if wn_tag not in (wn.VERB, wn.ADJ, wn.ADV):
                return 0
            else:
                return 1
        else:
            return 0
    else:
        if wn_tag not in (wn.NOUN, wn.VERB, wn.ADJ, wn.ADV):
                return 0
        else:
            return 1

def cleanData(text):
    txt = str(text)
    if txt == '[deleted]':
        return ''
    txt = re.sub(r'https|http|ftp|www?:\/\/.*[\r\n]*', ' ', txt, flags=re.MULTILINE)
    txt = re.sub("((@)|(#)[A-Za-z0-9]+)"," ",txt)
    txt = re.sub(r'\/s',r' ',txt)
    #txt = re.sub(r'[^A-Za-z0-9\s]',r'',txt)
    #txt = re.sub(r'\s',r' ',txt)
    return txt

def count_excl(txt):
    counter = 0
    for char in txt:
        if(char == '!'):
            counter += 1
    return counter

def count_ques(txt):
    counter = 0
    for char in txt:
        if(char == '?'):
            counter += 1
    return counter

def count_dots(txt):
    counter = 0
    for char in txt:
        if(char == '.'):
            counter += 1
    return counter

def count_quotes(txt):
    counter = 0
    for char in txt:
        if(char == '\'' or char == '"'):
            counter += 1
    return counter

def count_pw(tagged_sentence):
    counter = 0
    for word, tag in tagged_sentence:
        sentiment = sentiment_score(word, tag, 0)
        if(sentiment > 0):
            counter += 1;
    return counter

def count_nw(tagged_sentence):
    counter = 0
    for word, tag in tagged_sentence:
        sentiment = sentiment_score(word, tag, 0)
        if sentiment < 0:
            counter += 1;
    return counter

def count_PW(tagged_sentence):
    counter = 0
    for word, tag in tagged_sentence:
        counter += sentiment_score(word, tag, 1)
    return counter

def count_NW(tagged_sentence):
    counter = 0
    for word, tag in tagged_sentence:
        counter += sentiment_score(word, tag, 2)
    return counter

def count_P(P, N, p, n):
    if P==0 and N==0 and p==0 and n==0:
        return 0
    return ((delta*P + p) - (delta*N + n))/((delta*P + p) + (delta*N + n))

def count_rep(sent):
    for w in sent:
        i = 0
        length = len(w) - 3
        while i <= length:
            if w[i]==w[i+1] and w[i+1]==w[i+2]:
                return 1
            i+=1
    return 0

def count_ld(tagged_sentence):
    counter = 0
    for word, tag in tagged_sentence:
        counter += sentiment_score(word, tag, 3)
    return counter

def count_caps(sent):
    counter = 0
    for w in sent:
        flag = 0
        for char in w:
            if w<'A' or w>'Z':
                flag = 1
                break
        if flag == 0:
            counter += 1
    return counter

index_features_df = 0
def selectFeatures(x, isSarcasm):
    global index_features_df
    txt = str(x)
    len_txt = len(txt)
    excl = count_excl(txt)
    ques = count_ques(txt)
    dots = count_dots(txt)
    quotes = count_quotes(txt)
    
    cln_txt = txt
    cln_txt = re.sub(r'[^A-Za-z\s]',r' ',cln_txt) #cleaned text after removing ?,',",!,.
    cln_txt = re.sub("\*", ' ', cln_txt) #for reddit data
    cln_txt = cln_txt.replace('\r', '').replace('\n', '')#for reddit data only
    len_cln_txt = len(cln_txt)
    words = word_tokenize(cln_txt) #Tokenizing here and at a later step POS tagging
    stop_words = set(stopwords.words("english"))
    filtered_sentence = []
    for w in words:
        if w not in stop_words and len(w)>=3:
            filtered_sentence.append(w)
    #print(filtered_sentence) #tocheck
    if len(filtered_sentence) > 3:
        tagged_sentence = pos_tag(word_tokenize(cln_txt)) #tells whether noun, adjective, verb, adverb,...
        #print(tagged_sentence)
        body = cln_txt
        pw = count_pw(tagged_sentence)
        nw = count_nw(tagged_sentence)
        PW = count_PW(tagged_sentence)
        NW = count_NW(tagged_sentence)
        P = count_P(PW, NW, pw, nw)
        rep = count_rep(filtered_sentence)
        caps = count_caps(txt)
        score = isSarcasm
        features_df.loc[index_features_df] = [body, pw, nw, PW, NW, P, rep, excl, ques, dots, caps, \
                                              quotes, score]
        index_features_df += 1

        if index_features_df % 1000 == 0:
            print(index_features_df)
    



In [17]:
def main():
    #global index_features_df
    sarcasmReddit_df['body'] = sarcasmReddit_df['body'].map(lambda x: cleanData(x))
    sarcasmReddit_df['body'].replace('', np.nan, inplace=True)
    sarcasmReddit_df['body'].replace(' ', np.nan, inplace=True)
    sarcasmReddit_df.dropna(subset=['body'], inplace=True)
    sarcasmReddit_df['body'] = sarcasmReddit_df['body'].map(lambda x: selectFeatures(x, 1))

    seriousReddit_df['body'] = seriousReddit_df['body'].map(lambda x: cleanData(x))
    seriousReddit_df['body'].replace('', np.nan, inplace=True)
    seriousReddit_df['body'].replace(' ', np.nan, inplace=True)
    seriousReddit_df.dropna(subset=['body'], inplace=True)
    seriousReddit_df['body'] = seriousReddit_df['body'].map(lambda x: selectFeatures(x, -1))
    
    features_df.to_csv('../input/CheckingTwitter.csv')

if __name__ == "__main__":
    main()


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
